## Reannotating sigma70 promoters

In [1]:
from pathlib import Path
import sys
sys.path.append('bitome2')
sys.path.append('')

from bitome.core import Bitome
from bitome.ml import *

In [2]:
import pickle
with open('sigma_pssm_ecoli.txt', 'rb') as handle: 
    data = handle.read()   
d = pickle.loads(data)
M_prib = d['M_prib']
M_m35 = d['M_m35']
m10_seq = d['m10_seq']
m35_seq = d['m35_seq']
M_prib_sigma70 = {base: [pos_dict[base] for pos_dict in M_prib['Sigma70']] for base in 'ATCG'}
M_m35_sigma70 = {base: [pos_dict[base] for pos_dict in M_m35['Sigma70']] for base in 'ATCG'}

In [3]:
bitome_genbank = Bitome('NC_000913.3.gb')
tfbs = pd.read_csv('TFBS_regulondb.csv')
tfbs = tfbs.rename(columns={'Transcription_unit_id_regulated_by_TF': 'locus_tag'})
tu_table = pd.read_csv('tu.csv')
result_df = pd.merge(tfbs, tu_table, on='locus_tag', how='left')
result_df.to_csv('tfbs_tu_merged.csv')

In [4]:
a=[]
for i,j,k in zip(tu_table['sigma_factor'], tu_table['strand'], tu_table['tss']):
    if not isinstance(i, str):
        if k.is_integer():
            if j == 1:
                #print(k)
                m_10_left = (k + int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m10_location'])) -3
                m_10_right = (k + int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m10_location'])) +3
                m_35_left = (k + int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m35_location'])) -3
                m_35_right = (k + int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m35_location'])) +3
                #spacer length < 23
                if (-m_35_right + m_10_left) < 23:
                    a.append(['Sigma70', k, m_10_left, m_10_right, m_35_left, m_35_right])
            if j == -1:
                m_10_left = (k - int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m10_location'])) -3
                m_10_right = (k - int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m10_location'])) +3
                m_35_left = (k - int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m35_location'])) -3
                m_35_right = (k - int(bitome_genbank.featurize_promoter(k, j, minus_10_motif=M_prib_sigma70, minus_35_motif=M_m35_sigma70,
                                 usr=(-65, -45))['m35_location'])) +3
                #spacer length < 23
                if (-m_10_right + m_35_left) < 23:
                    a.append(['Sigma70', k, m_10_left, m_10_right, m_35_left, m_35_right])
            
print(len(a))

484


In [5]:
update_columns = ['sigma_factor', 'tss', 'box_10_left', 'box_10_right','box_35_left', 'box_35_right']

for sublist in a:
    match_value = sublist[1]
    matching_index = result_df[result_df['tss'] == match_value].index
    if not matching_index.empty:
        result_df.loc[matching_index, update_columns] = sublist[0:len(update_columns)]

result_df = result_df.set_index('locus_tag')
print(result_df)

             TF_identifier TF_name    Confornation_name  \
locus_tag                                                 
ECK120027580  ECK125328145    AaeR                 AaeR   
ECK120009421  ECK125286586    AccB                 AccB   
ECK120009381  ECK120015994    AcrR                 AcrR   
ECK120009328  ECK120015994    AcrR                 AcrR   
ECK120008981  ECK120015994    AcrR                 AcrR   
...                    ...     ...                  ...   
ECK120009822  ECK120014103    ZraR  ZraR-phosphorylated   
ECK120009038  ECK120012020     Zur  Zur-Zn<SUP>2+</SUP>   
ECK120009040  ECK120012020     Zur  Zur-Zn<SUP>2+</SUP>   
ECK120015950  ECK120012020     Zur  Zur-Zn<SUP>2+</SUP>   
ECK120026596  ECK120012020     Zur  Zur-Zn<SUP>2+</SUP>   

             TF_binding site_identifier  TF_bs_left_end_position  \
locus_tag                                                          
ECK120027580               ECK125258258                      NaN   
ECK120009421               E

In [6]:
result_df.to_csv('result_df_updated.csv')